This kernel is most about feature engineering, some in there are good some arent <br />
And there are some simple expainations about what I was doing. <br /> 
But I think this can be a simple reference to anyone who doesnt have idea how to hit > 0.7 AUC score(Public Score) <br />
I had to admit, most of my submissions are the blending result of my previous submission.(Since Im kinda lazy, and I just wanted to ran out the submission number) <br />
But the improvement was not very effectively. <br />
The real improvement was when I did the feature engineering seriouly. <br />

**Here are some place I think it is worth to try/modify/improve this kernel<br />**

1. Missing value filling of Distance
2. Encoding features with lable ( I had tried few times, it is pretty easy to overfit. But I think it might would be a breaking point )
3. Feature selection/dropping ( There are many similar features in it, some of them might be bad. But I guess some of the models like gradientBoostingTree,randomForest... should be handle this situation well )
4. Model tuning/selection ( Might can try stacking or blending, but this will take a lot of time to submit and this is why I was not doing it )

It is welcome to discuss ! Please feel free to point out my mistake! <br />
Also you can download this kernel from my github or from here directly. You can link to my github through my kaggle profile. Thanks, have a nice day.

# Modules Import

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../homework/data/midterm/"))
import copy
import datetime
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression
from mlxtend.regressor import StackingRegressor
import sklearn.metrics
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

['train_offline.csv', 'submit_1560351520.csv', 'column_description.csv', 'sample_submission.csv', 'test_offline.csv']


/Users/ken/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ModuleNotFoundError: No module named 'xgboost'

# Data Checking

In [ ]:
train_df = pd.read_csv('../homework/data/midterm/train_offline.csv')
test_df = pd.read_csv('../homework/data/midterm/test_offline.csv')

test_ids = test_df[['User_id','Coupon_id','Date_received']]

#Let's Check the shape of data first
print(f' training shape : {train_df.shape} ')
print(f' testing shape : {test_df.shape} ')

In [ ]:
def CheckMissingVals(data):
    for col in data.columns:
        if np.sum(data[col].isnull()) != 0:
            print(f' Missing values in {col} : {np.sum(data[col].isnull())}')

print("Count of missing data in training dataset: ")
CheckMissingVals(train_df)
print('\n')
print("Count of missing data in testing dataset: ")
CheckMissingVals(test_df)

# Deal with missing values

In [ ]:
total_df = pd.concat([train_df, test_df], axis = 0)

In [ ]:
DistanceFilling_UM = total_df.groupby(['User_id','Merchant_id'])['Distance'].mean().reset_index()
DistanceFilling_UM.columns = ['User_id','Merchant_id','DistanceFilling_UM']
DistanceFilling_U = total_df.groupby(['User_id'])['Distance'].mean().reset_index()
DistanceFilling_U.columns = ['User_id','DistanceFilling_U']
DistanceFilling_M = total_df.groupby(['Merchant_id'])['Distance'].mean().reset_index()
DistanceFilling_M.columns = ['Merchant_id','DistanceFilling_M']

total_df = pd.merge(total_df,DistanceFilling_UM,on = ['User_id','Merchant_id'], how = 'left')
total_df = pd.merge(total_df,DistanceFilling_U,on = ['User_id'], how = 'left')
total_df = pd.merge(total_df,DistanceFilling_M,on = ['Merchant_id'], how = 'left')
total_df.head(10)

In [ ]:
def DistanceMissingFill(data):
    if np.isnan(data['Distance']):
        if not np.isnan(data['DistanceFilling_UM']):
            return int(data['DistanceFilling_UM'])
        elif not np.isnan(data['DistanceFilling_U']):
            return int(data['DistanceFilling_U'])
        elif not  np.isnan(data['DistanceFilling_M']):
            return int(data['DistanceFilling_M'])
    return data['Distance']

total_df['Distance'] = total_df.apply(DistanceMissingFill, axis = 1)

In [ ]:
total_df = total_df.drop(['DistanceFilling_UM','DistanceFilling_U','DistanceFilling_M'], axis = 1)
train_df = total_df[:len(train_df)]
test_df = total_df[len(train_df):]

print("Count of missing data in training dataset: ")
CheckMissingVals(train_df)
print('\n')
print("Count of missing data in testing dataset: ")
CheckMissingVals(test_df)


**Our goal is to predict the use of Coupon, so obviously, if the row doesnt have coupon_id, we dont need to consider such condition.**

In [ ]:
train_df = train_df[~train_df.Coupon_id.isnull()]
test_df = test_df[~test_df.Coupon_id.isnull()]

print(f' training shape after dropping unwant rows: {train_df.shape} ')
print(f' testing shape after dropping unwant rows: {test_df.shape} ')

**Also we only analysis the use of Coupon within 15 days after user received the Coupon. So If there are some users didnt use the Coupon within 15 days, we consider they didnt use the Coupon, which the probability of using Coupon is zero.**

In [ ]:
#Dont forget to drop date column in testing data
test_df = test_df.drop(['Date'], axis = 1)

In [ ]:
def fifteenDaysChecking(data):
    if not np.isnan(data['Date']):
        #Means the user had used the coupon
        time_diff = pd.to_datetime(data['Date'], format = "%Y%m%d") - pd.to_datetime(data['Date_received'],
                                                                                   format = "%Y%m%d")
        if time_diff <= pd.Timedelta(15,'D'):
            return 1
    return 0
#This might will take a while
train_df['label'] = train_df.apply(fifteenDaysChecking, axis = 1)
print(train_df['label'].value_counts())
train_df.head(5)

In [ ]:
train_df = train_df.drop(['Date'], axis = 1)
#Check the missing values again
print("Count of missing data in training dataset: ")
CheckMissingVals(train_df)
print('\n')
print("Count of missing data in testing dataset: ")
CheckMissingVals(test_df)

In [ ]:
#Merge training & testing dataset for processing
train_label = train_df.label
train_df = train_df.drop(['label'], axis = 1)
total_df = pd.concat([train_df,test_df], axis = 0)
total_df.head(10)

In [ ]:
#Do a copy of original data , in case we do something wrong for the original data
temp_total_df = copy.deepcopy(total_df)

# Feature Engineering

In [ ]:
#Check the unique counts of each features
total_df.nunique()

## date features

In [ ]:
#Convert Date_received to str type for datetime processing
total_df['Date_received'] = total_df['Date_received'].astype('int').astype('str')
#Convert Date_received to other time information
total_df['Date_received'] = total_df['Date_received'].apply(lambda x : datetime.datetime.strptime(x,"%Y%m%d"))
total_df['Month_received'] = total_df['Date_received'].apply(lambda x : datetime.datetime.strftime(x,"%m")).astype("int64")
total_df['Day_received'] = total_df['Date_received'].apply(lambda x : datetime.datetime.strftime(x,"%d")).astype("int64")

total_df.head(10)

In [ ]:
total_df['Month_Cycle'] = total_df.Day_received.map(lambda x : 1 if x <= 15 else 0)

In [ ]:
# Record the total days to June
def set2June(data):
    if data['Month_received'] < 6:
        return (6-data['Month_received'])*30 - data['Day_received']
    return 1
        
total_df['CloseToJune'] = total_df.apply(set2June , axis = 1)


## Discount features

In [ ]:
#Let`s check what values in discount_rate feature
total_df.Discount_rate.value_counts()

**From discount_rate feautre, we can see that there are two types of discount. One of them is directly discount and another is the discount that user's consumption need to reach certain price.**

In [ ]:
total_df['DiscountType'] = total_df.Discount_rate.map(lambda x: 1 if (':' in x) else 0)
total_df.DiscountType.value_counts()

**I would like to add threes feature from DiscountType. <br />**
1. record what price need to reach that allow to get discount <br />
2. record when the price reach to discount, how much money can be discount <br />
3. record the discount ratio of both discount type <br />
4. record how much money we need to cost after reach the distcount bound

In [ ]:
total_df['DiscountBound'] = total_df.Discount_rate.map(lambda x: int(x.split(':')[0]) if (':' in x) else 0)
total_df['DirectPriceCut'] = total_df.Discount_rate.map(lambda x: int(x.split(':')[1]) if (':' in x) else 0)
total_df['DiscountRatio'] = total_df.Discount_rate.map(lambda x: (1 - float(x.split(':')[1])/float(x.split(':')[0])) if(':' in x) else float(x) )
total_df['MoneyCost'] = total_df['DiscountBound'] - total_df['DirectPriceCut']
total_df = total_df.drop(['Discount_rate'], axis = 1)
total_df.head(10)

**I would like to group the Discount upper bound**

In [ ]:
#Check the data distribution fitst
total_df['DiscountBound'].hist()
plt.title('Histogram of Discount Bound')
plt.xlabel('DiscountBound')
plt.ylabel('Count')
plt.show()

In [ ]:
cuttingArr = np.array([-1,80,150,250,total_df.DiscountBound.max()+1])
total_df['DiscountBound_Group'] = pd.cut(total_df.DiscountBound, cuttingArr)

print(total_df.DiscountBound_Group.value_counts())
print('\n')
#encoding intervals into integer
DiscountBound_intervals = total_df.DiscountBound_Group.unique()
total_df.DiscountBound_Group.replace(to_replace = DiscountBound_intervals[0],value = 0, inplace = True)
total_df.DiscountBound_Group.replace(to_replace = DiscountBound_intervals[1],value = 1, inplace = True)
total_df.DiscountBound_Group.replace(to_replace = DiscountBound_intervals[2],value = 2, inplace = True)
total_df.DiscountBound_Group.replace(to_replace = DiscountBound_intervals[3],value = 3, inplace = True)
print(total_df.DiscountBound_Group.value_counts())

## User id features

**I would like to use User_id feature to group with other features. I hope this can find some characteristic or behavior of user**

In [ ]:
Same_Merchant_User_received = total_df[['User_id','Merchant_id']]
Same_Merchant_User_received['temp'] = 1
Same_Merchant_User_received = Same_Merchant_User_received.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
Same_Merchant_User_received.columns = ['User_id','Merchant_id','Same_Merchant_User_received']

total_df = pd.merge(total_df, Same_Merchant_User_received, on = ['User_id','Merchant_id'], how = 'left')

**Record how many coupons each user received**

In [ ]:

Total_Coupon_User_received = total_df.groupby(['User_id'])['Coupon_id'].count().reset_index()
Total_Coupon_User_received.columns = ['User_id', 'Total_Coupon_User_received']
total_df = pd.merge(total_df,Total_Coupon_User_received, on = ['User_id'], how = 'left')

**Record how many same coupons each user received**

In [ ]:
Same_Coupon_User_received = total_df[['User_id', 'Coupon_id']]
Same_Coupon_User_received['temp'] = 1
Same_Coupon_User_received = Same_Coupon_User_received.groupby(['User_id','Coupon_id']).agg('sum').reset_index()
Same_Coupon_User_received.columns = ['User_id','Coupon_id','Same_Coupon_User_received']

total_df = pd.merge(total_df,Same_Coupon_User_received , on = ['User_id', 'Coupon_id'], how = 'left')

**Record how many same coupons each user received at the same day**

In [ ]:
Same_Day_Same_Coupon_User_received = total_df[['User_id','Coupon_id','Date_received']]
Same_Day_Same_Coupon_User_received['temp'] = 1
Same_Day_Same_Coupon_User_received = Same_Day_Same_Coupon_User_received.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()
Same_Day_Same_Coupon_User_received.columns = ['User_id','Coupon_id','Date_received','Same_Day_Same_Coupon_User_received']

total_df = pd.merge(total_df, Same_Day_Same_Coupon_User_received , on = ['User_id','Coupon_id','Date_received'], how = 'left')

**Record how many coupons each user received at the same day**

In [ ]:

Same_Day_Total_Coupon_User_received = total_df[['User_id','Date_received']]
Same_Day_Total_Coupon_User_received['temp'] = 1
Same_Day_Total_Coupon_User_received = Same_Day_Total_Coupon_User_received.groupby(['User_id','Date_received']).agg('sum').reset_index()
Same_Day_Total_Coupon_User_received.columns = ['User_id','Date_received', 'Same_Day_Total_Coupon_User_received']

total_df = pd.merge(total_df, Same_Day_Total_Coupon_User_received, on = ['User_id', 'Date_received'], how ='left')


**Record the mean/max/min discount upper bound of coupon that each user received**

In [ ]:

User_Received_Coupon_DistcountBound_Mean = total_df.groupby(['User_id'])['DiscountBound'].mean().reset_index()
User_Received_Coupon_DistcountBound_Mean.columns = ['User_id','User_Received_Coupon_DistcountBound_Mean']

User_Received_Coupon_DistcountBound_Max = total_df.groupby(['User_id'])['DiscountBound'].max().reset_index()
User_Received_Coupon_DistcountBound_Max.columns = ['User_id','User_Received_Coupon_DistcountBound_Max']

User_Received_Coupon_DistcountBound_Min = total_df.groupby(['User_id'])['DiscountBound'].min().reset_index()
User_Received_Coupon_DistcountBound_Min.columns = ['User_id','User_Received_Coupon_DistcountBound_Min']

total_df = pd.merge(total_df,User_Received_Coupon_DistcountBound_Mean, on = ['User_id'], how = 'left')
total_df = pd.merge(total_df,User_Received_Coupon_DistcountBound_Max, on = ['User_id'], how = 'left')
total_df = pd.merge(total_df,User_Received_Coupon_DistcountBound_Min, on = ['User_id'], how = 'left')


**Record the mean/max/min Money Cost of coupon that each user received**

In [ ]:

User_Received_Coupon_MoneyCost_Mean = total_df.groupby(['User_id'])['MoneyCost'].mean().reset_index()
User_Received_Coupon_MoneyCost_Mean.columns = ['User_id','User_Received_Coupon_MoneyCost_Mean']

User_Received_Coupon_MoneyCost_Max = total_df.groupby(['User_id'])['MoneyCost'].max().reset_index()
User_Received_Coupon_MoneyCost_Max.columns = ['User_id','User_Received_Coupon_MoneyCost_Max']

User_Received_Coupon_MoneyCost_Min = total_df.groupby(['User_id'])['MoneyCost'].min().reset_index()
User_Received_Coupon_MoneyCost_Min.columns = ['User_id','User_Received_Coupon_MoneyCost_Min']

total_df = pd.merge(total_df,User_Received_Coupon_MoneyCost_Mean, on = ['User_id'], how = 'left')
total_df = pd.merge(total_df,User_Received_Coupon_MoneyCost_Max, on = ['User_id'], how = 'left')
total_df = pd.merge(total_df,User_Received_Coupon_MoneyCost_Min, on = ['User_id'], how = 'left')



**Record the mean discount ratio of coupon that each user received**

In [ ]:
User_Received_Coupon_DiscountRatio_Mean = total_df.groupby(['User_id'])['DiscountRatio'].mean().reset_index()
User_Received_Coupon_DiscountRatio_Mean.columns = ['User_id','User_Received_Coupon_DiscountRatio_Mean']
total_df = pd.merge(total_df,User_Received_Coupon_DiscountRatio_Mean, on = ['User_id'], how = 'left')

**Record the minimal days that user received the same coupon. If the user only got the coupon once, then make the day gaps big.**

In [ ]:
#This will take a long~ time
def CalMinReceivedGap(data):

    dates = data.split(':')
    if len(dates) == 1:
        return -1 #Means only received once
    MinGap = 365
    for i in range(0,len(dates)-1):
        cur_gap = pd.to_datetime( dates[i] , format = "%Y-%m-%d") - pd.to_datetime(dates[i+1] , format = "%Y-%m-%d")
        cur_gap = abs(cur_gap.days)
        if cur_gap < MinGap:
                MinGap = cur_gap
    return MinGap

Min_Coupon_received_gap = total_df[['User_id','Coupon_id','Date_received']]
Min_Coupon_received_gap.Date_received = Min_Coupon_received_gap.Date_received.astype('str')
Min_Coupon_received_gap =  Min_Coupon_received_gap.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x : ':'.join(x)).reset_index()
Min_Coupon_received_gap['Min_Coupon_received_gap'] = Min_Coupon_received_gap.Date_received.apply(CalMinReceivedGap)
Min_Coupon_received_gap = Min_Coupon_received_gap.drop(['Date_received'], axis = 1)

total_df = pd.merge(total_df,Min_Coupon_received_gap,on = ['User_id','Coupon_id'], how ='left')
max_received_gap = total_df.Min_Coupon_received_gap.max()
total_df.Min_Coupon_received_gap = total_df.Min_Coupon_received_gap.map(lambda x : x if x != -1 else max_received_gap)

**Record the maximum days that user received the same coupon. If the user only got the coupon once, then make the day gaps big.**

In [ ]:
def CalMaxReceivedGap(data):
    dates = data.split(':')
    if len(dates) == 1:
        return -1 #Means only received once
    MaxGap = 0
    for i in range(0,len(dates)-1):
        cur_gap = pd.to_datetime( dates[i] , format = "%Y-%m-%d") - pd.to_datetime(dates[i+1] , format = "%Y-%m-%d")
        cur_gap = abs(cur_gap.days)
        if cur_gap > MaxGap:
                MaxGap = cur_gap
    return MaxGap


Max_Coupon_received_gap = total_df[['User_id','Coupon_id','Date_received']]
Max_Coupon_received_gap.Date_received = Max_Coupon_received_gap.Date_received.astype('str')
Max_Coupon_received_gap =  Max_Coupon_received_gap.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x : ':'.join(x)).reset_index()
Max_Coupon_received_gap['Max_Coupon_received_gap'] = Max_Coupon_received_gap.Date_received.apply(CalMaxReceivedGap)
Max_Coupon_received_gap = Max_Coupon_received_gap.drop(['Date_received'], axis = 1)

total_df = pd.merge(total_df,Max_Coupon_received_gap,on = ['User_id','Coupon_id'], how ='left')
max_received_gap = total_df.Max_Coupon_received_gap.max()
total_df.Max_Coupon_received_gap = total_df.Max_Coupon_received_gap.map(lambda x : x if x != -1 else max_received_gap)
total_df.head(10)

## Merchant features

**Here I will use Merchant_id feature to group with other features. This just vey similar to the processing of User_id feature.**

In [ ]:

Merchant_DiscountBound_mean = total_df.groupby(['Merchant_id'])['DiscountBound'].mean().reset_index()
Merchant_DiscountBound_mean.columns = ['Merchant_id','Merchant_DiscountBound_mean']

Merchant_DiscountBound_max = total_df.groupby(['Merchant_id'])['DiscountBound'].max().reset_index()
Merchant_DiscountBound_max.columns = ['Merchant_id','Merchant_DiscountBound_max']

Merchant_DiscountBound_min = total_df.groupby(['Merchant_id'])['DiscountBound'].min().reset_index()
Merchant_DiscountBound_min.columns = ['Merchant_id','Merchant_DiscountBound_min']

Merchant_DiscountRatio_max = total_df.groupby(['Merchant_id'])['DiscountRatio'].max().reset_index()
Merchant_DiscountRatio_max.columns = ['Merchant_id','Merchant_DiscountRatio_max']

Merchant_DiscountRatio_min = total_df.groupby(['Merchant_id'])['DiscountRatio'].min().reset_index()
Merchant_DiscountRatio_min.columns = ['Merchant_id','Merchant_DiscountRatio_min']

Merchant_DirectPriceCut_max = total_df.groupby(['Merchant_id'])['DirectPriceCut'].max().reset_index()
Merchant_DirectPriceCut_max.columns = ['Merchant_id','Merchant_DirectPriceCut_max']

Merchant_DirectPriceCut_min = total_df.groupby(['Merchant_id'])['DirectPriceCut'].min().reset_index()
Merchant_DirectPriceCut_min.columns = ['Merchant_id','Merchant_DirectPriceCut_min']

Merchant_DirectPriceCut_mean = total_df.groupby(['Merchant_id'])['DirectPriceCut'].mean().reset_index()
Merchant_DirectPriceCut_mean.columns = ['Merchant_id', 'Merchant_DirectPriceCut_mean']

Merchant_MoneyCost_mean = total_df.groupby(['Merchant_id'])['MoneyCost'].mean().reset_index()
Merchant_MoneyCost_mean.columns = ['Merchant_id', 'Merchant_MoneyCost_mean']

Merchant_MoneyCost_max = total_df.groupby(['Merchant_id'])['MoneyCost'].max().reset_index()
Merchant_MoneyCost_max.columns = ['Merchant_id', 'Merchant_MoneyCost_max']

Merchant_MoneyCost_min = total_df.groupby(['Merchant_id'])['MoneyCost'].min().reset_index()
Merchant_MoneyCost_min.columns = ['Merchant_id', 'Merchant_MoneyCost_min']

total_df = pd.merge(total_df,Merchant_DiscountBound_mean,on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_DirectPriceCut_mean,on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_MoneyCost_mean,on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_DiscountBound_max, on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_DiscountBound_min, on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_DiscountRatio_max, on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_DiscountRatio_min, on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_DirectPriceCut_max, on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_DirectPriceCut_min, on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_MoneyCost_max, on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_MoneyCost_min, on = ['Merchant_id'], how = 'left')

In [ ]:
Mcount = total_df.Merchant_id.value_counts().reset_index()
Mcount.columns = ['Merchant_id','Merchant_count']
Ccount = total_df.Coupon_id.value_counts().reset_index()
Ccount.columns = ['Coupon_id','Coupon_count']

total_df = pd.merge(total_df,Mcount,on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Ccount,on = ['Coupon_id'], how = 'left')



**Let check the correlation between label and features to judge what we just did is good or not**

In [ ]:
#Check the correlation again before fill the missing values of Distance
correlation = copy.deepcopy(total_df[:len(train_df)])
correlation = pd.concat([correlation, pd.DataFrame(train_label.values, columns = ['label'], index = correlation.index)], axis = 1)
corr = correlation.corr()
plt.figure(figsize = (30,30))
sns.heatmap(corr, cmap = plt.cm.summer, annot = True)
plt.title('Correlation of training dataset')
plt.show()

## Distance features

In [ ]:
#There are still some Distance values missing
#Let's check the ratio between missing values and non-missing values

print(f' Missing ratio of Distance : {np.sum(total_df.Distance.isnull()) / len(total_df)}')

I have a bold idea, the distance might will have little correlation of when user received the coupon. Since the merchant might would send the coupon in certain days. So If there are some users received the coupon at similar time, they might had got the coupon at the same merchant. Which means it has slightly chance that these users have similar event location. 

In [ ]:
total_df['Distance'] = total_df.groupby(['CloseToJune'])['Distance'].transform(lambda x : x.fillna(x.mode()[0]))
CheckMissingVals(total_df)
total_df.head(10)

In [ ]:
User_Activity_Distance_Mean = total_df.groupby(['User_id'])['Distance'].mean().reset_index()
User_Activity_Distance_Mean.columns = ['User_id','User_Activity_Distance_Mean']
total_df = pd.merge(total_df,User_Activity_Distance_Mean,on = ['User_id'], how = 'left')

User_Activity_Distance_Max = total_df.groupby(['User_id'])['Distance'].max().reset_index()
User_Activity_Distance_Max.columns = ['User_id','User_Activity_Distance_Max']
total_df = pd.merge(total_df,User_Activity_Distance_Max,on = ['User_id'], how = 'left')

User_Activity_Distance_Min = total_df.groupby(['User_id'])['Distance'].min().reset_index()
User_Activity_Distance_Min.columns = ['User_id','User_Activity_Distance_Min']
total_df = pd.merge(total_df,User_Activity_Distance_Min,on = ['User_id'], how = 'left')


In [ ]:
total_df['Bound_Distance'] = total_df['DiscountBound']+1 * (total_df['Distance']+1)
total_df['DiscountRatio_Distance'] = total_df['DiscountRatio'] * (total_df['Distance']+1)
total_df['MoneyCost_Distance'] = total_df['MoneyCost']+1 * (total_df['Distance']+1)
total_df['DirectPriceCut_Distance'] = total_df['DirectPriceCut']+1 / (total_df['Distance']+1) 


In [ ]:
Merchant_Distance_mean = total_df.groupby(['Merchant_id'])['Distance'].mean().reset_index()
Merchant_Distance_mean.columns = ['Merchant_id','Merchant_Distance_mean']

Merchant_Distance_max = total_df.groupby(['Merchant_id'])['Distance'].max().reset_index()
Merchant_Distance_max.columns = ['Merchant_id','Merchant_Distance_max']

Merchant_Distance_min = total_df.groupby(['Merchant_id'])['Distance'].min().reset_index()
Merchant_Distance_min.columns = ['Merchant_id','Merchant_Distance_min']


Coupon_Distance_mean = total_df.groupby(['Coupon_id'])['Distance'].mean().reset_index()
Coupon_Distance_mean.columns = ['Coupon_id', 'Coupon_Distance_mean']

Coupon_Distance_max = total_df.groupby(['Coupon_id'])['Distance'].max().reset_index()
Coupon_Distance_max.columns = ['Coupon_id', 'Coupon_Distance_max']

Coupon_Distance_min = total_df.groupby(['Coupon_id'])['Distance'].min().reset_index()
Coupon_Distance_min.columns = ['Coupon_id', 'Coupon_Distance_min']

In [ ]:
total_df = pd.merge(total_df,Merchant_Distance_mean,on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_Distance_max,on = ['Merchant_id'], how = 'left')
total_df = pd.merge(total_df,Merchant_Distance_min,on = ['Merchant_id'], how = 'left')

total_df = pd.merge(total_df,Coupon_Distance_mean, on = ['Coupon_id'], how = 'left')
total_df = pd.merge(total_df,Coupon_Distance_max, on = ['Coupon_id'], how = 'left')
total_df = pd.merge(total_df,Coupon_Distance_min, on = ['Coupon_id'], how = 'left')


## Features encoding with label

In [ ]:
'''
trainSet = copy.deepcopy(total_df[:len(train_df)])
trainSet = pd.concat([trainSet, pd.DataFrame(train_label.values, columns = ['label'], index = trainSet.index)], axis = 1)
trainSet.head(3)
'''

In [ ]:
'''
Distance_Accepted_rate = trainSet.groupby(['Distance'])['label'].mean().reset_index()
Distance_Accepted_rate.columns = ['Distance','Distance_Accepted_rate']

DiscountBound_Accepted_rate = trainSet.groupby(['DiscountBound'])['label'].mean().reset_index()
DiscountBound_Accepted_rate.columns = ['DiscountBound', 'DiscountBound_Accepted_rate']

total_df = pd.merge(total_df, Distance_Accepted_rate , on = ['Distance'], how = 'left')
total_df = pd.merge(total_df, DiscountBound_Accepted_rate, on = ['DiscountBound'], how = 'left')

total_df.head(5)
'''

## Scale the features

In [ ]:
total_df = total_df.drop(['Merchant_id','Coupon_id','Day_received','Month_received','User_id','Date_received'],axis = 1)

In [ ]:

for col in total_df.columns:
    if (col != 'Distance_Accepted_rate') & (col != 'DiscountBound_Accepted_rate'):
        total_df[col] = MinMaxScaler().fit_transform(total_df[col].values.reshape(-1,1))        
total_df.head(10)

In [ ]:
#One hot encoding the discount type
total_df = pd.get_dummies(total_df, columns = ['DiscountType'], prefix = "DiscountType")
print(f' final shape of total data : {total_df.shape} ')

In [ ]:
#Check the correlation again before fill the missing values of Distance
correlation = copy.deepcopy(total_df[:len(train_df)])
correlation = pd.concat([correlation, pd.DataFrame(train_label.values, columns = ['label'], index = correlation.index)], axis = 1)
corr = correlation.corr()
plt.figure(figsize = (30,30))
sns.heatmap(corr, cmap = plt.cm.summer, annot = True)
plt.title('Correlation of training dataset')
plt.show()

# Model training & validation

In [ ]:
train_x = total_df[:len(train_df)]
test_x = total_df[len(train_df):]

def tuneParamsRandom(classifier, params, train_x, train_y, cv = 5):
    rs = RandomizedSearchCV(classifier, params, n_iter = 20, scoring = 'roc_auc', n_jobs = -1, verbose = 0, cv=cv)
    rs.fit(train_x, train_y)
    
    return rs.best_params_, abs(rs.best_score_)

In [ ]:

etParams = {'n_estimators':np.arange(100,1100,50), 'max_depth':np.arange(3,11,2),
            'min_samples_leaf':np.arange(1,6,2) }

rfParams = {'n_estimators':np.arange(100,1100,50), 'max_depth':np.arange(3,11,2),
            'min_samples_split':np.arange(2,20,2), 'min_samples_leaf':np.arange(1,6,2) }

abParams = {'n_estimators':np.arange(100,1100,50),'learning_rate':np.arange(0.01,0.2,0.05)}


lgbmcParams = {'n_estimators' : np.arange(400,1200,100), 'learning_rate' : np.arange(0.01,0.1,0.02),
               'num_leaves' : np.arange(2,48,4), 'max_depth' : np.arange(3,10,2),
               'subsample' : np.arange(0.3,0.8,0.1) }

gbcParams = {'n_estimators' : np.arange(400,1200,100) , 'learning_rate' : np.arange(0.01,0.1,0.02) 
             , 'min_samples_split' : np.arange(2,30,5), 'min_samples_leaf' : np.arange(2,32,4),
              'max_depth' : np.arange(3,8,2), 'subsample' : np.arange(0.3,0.8,0.1)}

xgbParams = {'max_depth':np.arange(3,8,1),'learning_rate':np.arange(0.01,0.1,0.02)
             ,'n_estimator': np.arange(1000,3000,100),'gamma':np.arange(0.01,0.1,0.02)}

lrParams = {'C':np.arange(0.01,1,0.05), 'max_iter' : np.arange(100,500,100)}

In [ ]:
train_label.index = train_x.index
tuneSet = pd.concat([train_x,train_label], axis = 1)
trainSet = tuneSet.sample(frac=0.5)
trainSet.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(trainSet[trainSet.columns[trainSet.columns != 'label']],trainSet['label'], test_size = 0.3, random_state = 1234)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

## LogisticRegression

In [ ]:
'''
lr = LogisticRegression(random_state = 2019,n_jobs = -1)
lr_best_Params, lr_best_score = tuneParamsRandom(lr,lrParams,x_train,y_train)
print("LogisticRegression :",lr_best_Params,lr_best_score)
lr = LogisticRegression(**lr_best_Params,n_jobs = -1)
lr.fit(x_train,y_train)
lr_pred_val = lr.predict_proba(x_test)
auc_score = roc_auc_score(y_true = y_test, y_score = lr_pred_val[:,1])
acc = accuracy_score(y_true = y_test, y_pred = lr_pred_val.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))
lr = LogisticRegression(**lr_best_Params,n_jobs = -1,random_state = 2019)
lr.fit(train_x, train_label)
lr_pred = lr.predict_proba(test_x)[:,1]
'''

## GradientBoostingClassifier

In [ ]:
'''
gbc = GradientBoostingClassifier(max_features = 'sqrt')
gbc_best_Params,gbc_best_score = tuneParamsRandom(gbc,gbcParams,x_train,y_train)
print("GradientBoostingClassifier:",gbc_best_Params,gbc_best_score)


gbc = GradientBoostingClassifier(**gbc_best_Params,max_features = 'sqrt')
gbc.fit(train_x, train_label)
gbc_pred = gbc.predict_proba(test_x)[:,1]

importances = pd.DataFrame(gbc.feature_importances_, columns = ['importances'], index = train_x.columns)
importances.importances.sort_values(ascending = False)
'''

## LGBMClassifier

In [ ]:

lgbmc_best_Params, lgbmc_best_score = tuneParamsRandom(LGBMClassifier(),lgbmcParams,x_train,y_train)
print("LGBMClassifier:",lgbmc_best_Params,lgbmc_best_score)


lgbmc = LGBMClassifier(**lgbmc_best_Params)
lgbmc.fit(train_x,train_label)
lgbmc_pred = lgbmc.predict_proba(test_x)[:,1]


## XGBClassifier

In [ ]:
'''
xgb_best_Params,xgb_best_score = tuneParamsRandom(XGBClassifier(),xgbParams, x_train, y_train)
print("XGBClassifier:",xgb_best_Params,xgb_best_score)

xgb = XGBClassifier(**xgb_best_Params)
xgb.fit(train_x,train_label)
xgb_pred = xgb.predict_proba(test_x)[:,1]
'''

## AdaBoost

In [ ]:
'''
ab_best_Params,ab_best_score = tuneParamsRandom(AdaBoostClassifier(),abParams,val_x,val_y)
print("AdaBoost:",ab_best_Params,ab_best_score)
'''

## RandomForest

In [ ]:
'''
rf_best_Params,rf_best_score = tuneParamsRandom(RandomForestClassifier(),rfParams,val_x,val_y)
print("RandomForest:",rf_best_Params,rf_best_score)
'''

## Blending

In [ ]:
#blending_pred = gbc_pred * 0.3 + lgbmc_pred * 0.7 

## Stacking

In [ ]:
'''
from mlxtend.classifier import StackingClassifier
gbc = GradientBoostingClassifier(**gbc_best_Params)
lgbmc = LGBMClassifier(**lgbmc_best_Params)
xgb = XGBClassifier(**xgb_best_Params)

meta_estimator = LGBMClassifier()
stacking = StackingClassifier(classifiers = [gbc,lgbmc, xgb], meta_classifier = meta_estimator)
stacking.fit(train_x,train_label)
stacking_pred = stacking.predict_proba(test_x)[:,1]
'''

# Submission

In [ ]:
test_ids = test_ids.loc[~test_ids.Coupon_id.isna()]

test_ids = pd.concat([test_ids, pd.DataFrame(lgbmc_pred, columns = ['label'], index = test_ids.index)] , axis = 1)

test_ids.loc[:,"User_id"] = test_ids["User_id"].apply(lambda x:str(int(x)))
test_ids.loc[:,"Coupon_id"] = test_ids["Coupon_id"].apply(lambda x:str(int(x)))
test_ids.loc[:,"Date_received"] = test_ids["Date_received"].apply(lambda x: str(int(x)))

test_ids["uid"] = test_ids[['User_id',"Coupon_id","Date_received"]].apply(lambda x: '_'.join(x.values), axis = 1)
test_ids.reset_index(drop = True, inplace = True)

In [ ]:
submission = test_ids.groupby("uid", as_index = False).mean()
submission = submission[["uid","label"]]
submission.to_csv("v9.csv", header = ["uid", "label"], index = False)

submission.head(10)